In [ ]:
!pip install transformers peft torch accelerate safetensors

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.6 MB/s eta 0:00:00


In [ ]:
! pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 57.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
)

from accelerate import Accelerator
from datasets import load_dataset
import pandas as pd
import numpy as np
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from peft import PeftModel, PeftConfig
import re
from sklearn import metrics
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import time

In [ ]:
# Step 1: Path to your adapter model and configuration
adapter_model_path = "/content/adapter_model.safetensors"
adapter_config_path = "/content/adapter_config.json"


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

access_token = "hf_XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your Hugging Face access token
base_model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pad_length = 600

# Load the tokenizer and model with the token
tokenizer = AutoTokenizer.from_pretrained(base_model_name, token=access_token, padding_side="right", add_eos_token=False, add_bos_token=True, max_length=pad_length, use_fast=False)
tokenizer.pad_token_id = 128001



tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
d = {0:"Satisfaction", 1:"Feasibility", 2:"Consistency", 3:"Completeness", 4:"Trust", 5:"Understandability", 6:"Fairness", 7:"Complexity"}
sys = """You are evaluating counterfactual explanations generated by AI.  Counterfactual explanations explain what parameters of a situation should have been different for the outcome to have been different. You are not expected to provide reasoning or explanation and should answer with the appropriate value from the set ["low", "medium", "high"]."""
sysend =  "The following is the counterfactual explanation. "

In [ ]:
person = "answer"

In [ ]:
def match1(i):
    if i==0:
        return " Please rate as 'low' (very unsatisfied), 'medium' or 'high' (completely satisfied), how satisfied would you be with such an explanation: "
    if i==1:
        return " Please rate as 'low' (very unfeasible), 'medium' or 'high' (completely feasible), how feasible is this explanation: "
    if i==2:
        return " Please rate as 'low' (very unconsistent), 'medium' or 'high' (completely consistent), how consistent is this explanation: "
    if i==3:
        return " Please rate as 'low' (very incomplete), 'medium' or 'high' (fully complete), how complete is this explanation: "
    if i==4:
        return " Please rate as 'low' (not at all), 'medium' or 'high' (completely), how much would you trust this explanation: "
    if i==5:
        return " Please rate as 'low' (incomprehensible), 'medium' or 'high' (very understandable), how understandable is this explanation: "
    if i==6:
        return " Please rate as 'low' (completely biased), 'medium' or 'high' (completely fair), how fair is this explanation: "
    if i==7:
        return " Please rate as 'low' (too easy), 'medium' (ideally complex) or 'high' (too complex), how complex is this explanation: "
    return " Please rate as 'low' (very unsatisfied), 'medium' or 'high' (completely satisfied), how satisfied would you be with such an explanation: "

def match2(i):
    if i==0:
        return " The definition of satisfaction: this scenario effectively explains how to reach a different outcome. "
    if i==1:
        return " The definition of feasibility: the actions suggested by the explanation are practical, realistic to implement and actionable. "
    if i==2:
        return " The definition of consistency: the parts of the explanation do not contradict each other. "
    if i==3:
        return " The definition of completeness: the explanation is sufficient in explaining how to achieve the desired outcome. "
    if i==4:
        return " The definition of trust: I believe that the suggested changes would bring about the desired outcome. "
    if i==5:
        return " The definition of understandability:  I feel like I understood the phrasing of the explanation well. "
    if i==6:
        return " The definition of fairness: the explanation is unbiased towards different user groups and does not operate on sensitive features. "
    if i==7:
        return " The definition of complexity: the explanation has an appropriate level of detail and complexity - not too simple, yet not overly complex."
    return " The definition of satisfaction: this scenario effectively explains how to reach a different outcome. "

In [ ]:
def prompt(s,i,answer):
    global sys
    global sysend
    prompt = s + match1(i)
    if(answer!=None):
        final = [
                    {"role":"system", "content":sys + match2(i) + sysend},
                    {"role":"user", "content":prompt},
                    {"role":"assistant", "content": answer}
                ]
    else:
        final = [
                    {"role":"system", "content":sys + match2(i) + sysend},
                    {"role":"user", "content":prompt}
                ]
    return final

In [ ]:
def preprocess_function(example, train):
    if(train):
        p = prompt(example["prompt"], example["type"], example[person])
        pre_token = tokenizer.apply_chat_template(p, tokenize=False, add_generation_prompt=False)
        example = tokenizer(pre_token, padding="max_length", max_length=pad_length, return_attention_mask=True)
    else:
        p = prompt(example["prompt"], example["type"], None)
        pre_token = tokenizer.apply_chat_template(p, tokenize=False, add_generation_prompt=True)
        example = tokenizer(pre_token, padding=False)

    example["prompt"] = pre_token
    return example

In [ ]:
data = load_dataset("csv", data_files={"test":"/content/test_dataset_200_questionsplit.csv"})

#data["train"] = data["train"].map(preprocess_function, fn_kwargs={"train":True} , batched=False)
data["test"] = data["test"].map(preprocess_function, fn_kwargs={"train":False} , batched=False)

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [ ]:
accelerator = Accelerator()

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto",
    token=access_token
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
adapter_model_path = "/content/adapter_model.safetensors"
adapter_config_path = "/content/adapter_config.json"

In [ ]:
# Load LoRA adapter config
peft_config = PeftConfig.from_pretrained(
    model,
    model_id=adapter_directory,  # Path to the directory containing both adapter_config.json and adapter_model.safetensors
)

In [ ]:
model = PeftModel.from_pretrained(model, model_id="/content/")

In [ ]:
# Evaluation loop
accuracy_scores = []
print("Starting testing")
current = time.time()
outps = []
evaluation_runs = 1

for j in range(evaluation_runs):  # Run multiple evaluation rounds
    results = []
    for i, pr in enumerate(data["test"]["prompt"]):
        print("Running prompt " + str(i) + "/" + str(len(data["test"])))
        inputs = tokenizer(pr, return_tensors="pt").to("cuda")
        model_output = model.generate(**inputs, temperature=1, do_sample=True, max_new_tokens=60)
        results.append([tokenizer.batch_decode(model_output, skip_special_tokens=True), data["test"][person][i], data["test"]["type"][i]])

    for i, (x, y, z) in enumerate(results):
        an = re.search("high|medium|low", x[0].split("assistant")[1].lower())
        result = "Failed"
        if an is not None:
            result = an.group()
        results[i].append(result)

    answe = np.array(results, dtype=object)

    # Confusion matrix and accuracy calculation
    confusion_matrix = metrics.confusion_matrix(answe[:, 1], answe[:, 3], labels=["low", "medium", "high"])
    print(confusion_matrix)

    acc = metrics.accuracy_score(answe[:, 1], answe[:, 3])
    accuracy_scores.append(acc)

    outps = outps + results

print("Type: " + person)
print(accuracy_scores)
print("Average time per answer: " + str((time.time() - current) / (evaluation_runs * len(data["test"]))))



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Starting testing
Running prompt 0/48


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Running prompt 1/48


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Running prompt 45/48


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Running prompt 46/48


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Running prompt 47/48
[[14  1  0]
 [ 0 16  2]
 [ 0  0 15]]
Type: answer
[0.9375]
Average time per answer: 2.1717787981033325


In [ ]:
# Save results to CSV
pd.DataFrame(np.array(outps, dtype=object)).to_csv("results_200_llama_eval.csv")